In [3]:
from pyspark.sql import SparkSession 
from pyspark.ml.feature import Word2Vec,Tokenizer,StopWordsRemover,NGram
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("SparkML Depression Analysis").getOrCreate()
dataDF= spark.read.option("inferSchema","True")\
                        .option("delimiter","\t")\
                        .csv("datasets/movie_turkish_train.txt")

dataDF = dataDF.withColumnRenamed("_c0","tweet")
dataDF = dataDF.withColumnRenamed("_c1","label")


def removeCharacters(text):
    text = text.replace(","," ")
    text = text.replace("."," ")
    text = text.replace("!"," ")
    text = text.replace("?"," ")
    text = text.replace("cok","çok")
    return text
    
removeUDF = UserDefinedFunction(removeCharacters,StringType())
dataDF = dataDF.withColumn("text",removeUDF('tweet'))

tokenizer = Tokenizer(inputCol='text',outputCol='tokens')
dataDF = tokenizer.transform(dataDF)


remover = StopWordsRemover(inputCol='tokens',outputCol='no-stop-words',stopWords=StopWordsRemover.loadDefaultStopWords('turkish'))
dataDF = remover.transform(dataDF)


ng = NGram(n=2,inputCol="no-stop-words",outputCol="ngrams")
dataDF = ng.transform(dataDF)

dataDF.show()


vSize = 400
w2v = Word2Vec(inputCol='ngrams',outputCol='features',vectorSize=vSize,windowSize=23,minCount=5)
dataDF = w2v.fit(dataDF).transform(dataDF)

dataDF.show()


dataDF = dataDF.select('features','label')
dataDF.printSchema()

trainDF,testDF = dataDF.randomSplit([0.75,0.25],seed=123)


mlpClassifier = MultilayerPerceptronClassifier(layers=[vSize,2,2])
model = mlpClassifier.fit(trainDF)


resultDF = model.transform(testDF) #Prediction
#resultDF.show(100)

eva = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = eva.evaluate(resultDF)
print("Test Accuracy : ",accuracy)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|               tweet|label|                text|              tokens|       no-stop-words|              ngrams|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|bunu da gördük uz...|    0|bunu da gördük uz...|[bunu, da, gördük...|[bunu, gördük, uz...|[bunu gördük, gör...|
|filmi begenmeyenl...|    0|filmi begenmeyenl...|[filmi, begenmeye...|[filmi, begenmeye...|[filmi begenmeyen...|
|günde 1 film izle...|    0|günde 1 film izle...|[günde, 1, film, ...|[günde, 1, film, ...|[günde 1, 1 film,...|
|afisine bakip ald...|    0|afisine bakip ald...|[afisine, bakip, ...|[afisine, bakip, ...|[afisine bakip, b...|
|sadece insanlarin...|    0|sadece insanlarin...|[sadece, insanlar...|[sadece, insanlar...|[sadece insanlari...|
|ucuz bir aksiyon ...|    0|ucuz bir aksiyon ...|[ucuz, bir, aksiy...|[ucuz, bir, aksiy...|[ucuz